https://keras.io/examples/nlp/neural_machine_translation_with_transformer/

# Setup

In [ ]:
# !pip install tensorflow==2.10

In [1]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [2]:
tf.__version__

'2.10.0'

#Downloading the data

In [3]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2638744/2638744 [==============================] - 0s 0us/step


# Parsing the data

In [4]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [5]:
for _ in range(5):
    print(random.choice(text_pairs))

('Students should make use of the books in the library.', '[start] Los estudiantes deberían utilizar los libros de la biblioteca. [end]')
('Maybe Tom likes swimming.', '[start] Quizá a Tom le guste nadar. [end]')
('Tom climbed down from the roof.', '[start] Tom se bajó del techo. [end]')
('Do you want a few more minutes?', '[start] ¿Quieres unos minutos más? [end]')
('Tom asked me if I knew anybody who could translate from French into English.', '[start] Tom me preguntó si conocía a alguien que pudiera traducir de francés a inglés. [end]')


In [6]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


# Vectorizing the text data

In [23]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [24]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [25]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [10]:
for inputs, targets in val_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


# Building the model

In [26]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [27]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

In [28]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [29]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [30]:
# embed_dim = 256
# latent_dim = 2048
# num_heads = 8

# encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
# print(encoder_inputs.shape)
# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# print(x.shape)
# encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
# print(encoder_outputs.shape)
# encoder = keras.Model(encoder_inputs, encoder_outputs)

# decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
# print(encoder_inputs.shape)
# encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
# print(encoded_seq_inputs.shape)
# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# print(x.shape)
# x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
# print(x.shape)
# x = layers.Dropout(0.5)(x)
# print(x.shape)
# decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
# print(decoder_outputs.shape)
# decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

# decoder_outputs = decoder([decoder_inputs, encoder_outputs])
# transformer = keras.Model(
#     [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
# )

# Training our model


In [33]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_2 (Positi  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_1 (Transfo  (None, None, 256)   3155456     ['positional_embedding_

In [34]:
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 102s 73ms/step - loss: 1.7274 - accuracy: 0.3962 - val_loss: 1.3959 - val_accuracy: 0.4781
Epoch 2/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.3996 - accuracy: 0.5078 - val_loss: 1.2274 - val_accuracy: 0.5508
Epoch 3/30
1302/1302 [==============================] - 94s 72ms/step - loss: 1.2370 - accuracy: 0.5626 - val_loss: 1.1216 - val_accuracy: 0.5864
Epoch 4/30
1302/1302 [==============================] - 93s 71ms/step - loss: 1.1414 - accuracy: 0.5964 - val_loss: 1.0837 - val_accuracy: 0.6072
Epoch 5/30
1302/1302 [==============================] - 92s 71ms/step - loss: 1.0924 - accuracy: 0.6186 - val_loss: 1.0546 - val_accuracy: 0.6196
Epoch 6/30
1302/1302 [==============================] - 93s 71ms/step - loss: 1.0641 - accuracy: 0.6338 - val_loss: 1.0473 - val_accuracy: 0.6257
Epoch 7/30
1302/1302 [==============================] - 93s 72ms/step - loss: 1.0434 - accuracy: 0.6457 - val_loss: 1.0379 

In [ ]:
train_ds

In [37]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print(input_sentence,translated)

Do you want your money in coins? [start] quieres tu dinero en el [UNK] [end]
Did he learn Japanese as well? [start] Él hizo japoneses y tu blanco como [UNK] [end]
Tom shouldn't have driven Mary's car. [start] tom no deberías haber yo el auto [UNK] de mary [end]
Thank you. [start] lo gracias [end]
Do you think I'm beautiful? [start] tú crees que soy bella [end]
I'm really worried about my child. [start] estoy realmente preocupado por mi hijo [end]
Tom eats nothing but fruit. [start] tom no come nada que yo [UNK] [end]
That hurts. [start] tom duele [end]
This question is very simple. [start] esta pregunta es muy [UNK] [end]
This is the best ink. [start] este es la mejor ropa [end]
I told you not to move. [start] te dije que no se [UNK] [end]
Do I have to do this now? [start] tengo que hacerlo ahora [end]
This lake is among the deepest in the country. [start] este lago es entre el más [UNK] el país [end]
I'll see you at 2:30. [start] te veré en las dos y media [end]
This isn't my umbrella

In [36]:
translated

'[start] por qué está usted un suéter [end]'